### Different LSTM and autoencoder approaches for encoding and k-means for clustering (MIT-BIH ECG data)

Inspiration from:
* https://github.com/navneetkr123/Clustering-using-deep-learning-LSTM-Autoencoder-Kmeans-/blob/master/switchon_new.ipynb

* https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

* https://blog.keras.io/building-autoencoders-in-keras.html

In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.cluster import KMeans
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed
from keras.models import Model, Sequential

import matplotlib.pyplot as plt
import umap.umap_ as umap

#### Data Loader and Preprocessing

In [ ]:
train = np.loadtxt('data/mitbih_train.csv', delimiter=",",dtype=float)
test = np.loadtxt('data/mitbih_test.csv',delimiter=",",dtype=float)

In [ ]:
np.random.shuffle(train)
np.random.shuffle(test)

print("original size train: ", train.shape)
print("original size test: ", test.shape)

#reduce nr. of samples
train_small = train[:-77554]
test_small = test[:-19844]

print("small size train: ", train_small.shape)
print("small size test: ", test_small.shape)

#split to X and y
X_train = train_small[:, 0:-1]
y_train = train_small[:, -1]

X_test = test_small[:, 0:-1]
y_test = test_small[:, -1]

##### create irregular TS

In [ ]:
#check number of zeros in each row in original data (already irregular)
n_zeros = np.count_nonzero(X_train==0, axis=1)
print(n_zeros)

In [ ]:
# TRAIN DATA
# random boolean mask for which values will be changed
replace_rate = 0.3
mask = np.random.choice([0, 1], size=X_train.shape, p=((1-replace_rate),replace_rate)).astype(np.bool)

# random matrix the same shape
r = np.zeros(shape=X_train.shape)
#r = np.full(shape=X_train.shape, fill_value=-1, dtype=np.float32)

# use  mask to replace values in input array
X_train[mask] = r[mask]


# TEST DATA
mask_test = np.random.choice([0, 1], size=X_test.shape, p=((1-replace_rate),replace_rate)).astype(np.bool)
r_test = np.zeros(shape=X_test.shape)
X_test[mask_test] = r_test[mask_test]

In [ ]:
n_zeros = np.count_nonzero(X_train==0, axis=1)
for i in n_zeros:
    print(i)

In [ ]:
train = pd.DataFrame(X_train)
test = pd.DataFrame(X_test)
train.head(3)

In [ ]:
train = train.values
test = test.values

train.shape

### Clustering Algorithms

#### SKlearn K-Means

In [ ]:
def sklearnkmeans(encoded_data):
    kmeans = KMeans(n_clusters=5).fit(encoded_data)
    labels = kmeans.predict(encoded_data)

    centroids = kmeans.cluster_centers_

    return centroids, labels
    

#### DTW K-Means

In [ ]:
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

def k_means_dtw(data,num_clust,num_iter,w=5):
    centroids=random.sample(list(data),num_clust)
    counter=0
    for n in range(num_iter):
        counter+=1
        print(counter)
        assignments={}
        labels = []

        #assign data points to clusters
        for ind,i in enumerate(data):
            min_dist=float('inf')
            closest_clust=None
            for c_ind,j in enumerate(centroids):
                cur_dist=DTWDistance(i,j,w)
                if cur_dist<min_dist:
                    min_dist=cur_dist
                    closest_clust=c_ind
            if closest_clust in assignments:
                assignments[closest_clust].append(ind)
            else:
                assignments[closest_clust]=[]
            labels.append(closest_clust)
    
        #recalculate centroids of clusters
        for key in assignments:
            clust_sum=0
            for k in assignments[key]:
                clust_sum=clust_sum+data[k]
            centroids[key]=[m/len(assignments[key]) for m in clust_sum]
    
    return centroids, labels

#### UMAP

 Uniform Manifold Approximation and Projection (UMAP) -  projects the high-dimensional data points into 2D/3D by inducing the projected data to have a similar distribution as the original data points by minimizing something called the KL divergence.
 
there are still valid reasons to use UMAP as a preprocessing step for clustering. As with any clustering approach one will want to do some exploration and evaluation of the clusters that come out to try to validate them if possible.

In [ ]:
def umapt(data):
    umap_2d = umap.UMAP(n_components=2, init='random', random_state=42)
    #umap_3d = UMAP(n_components=3, init='random', random_state=0)

    proj_2d = umap_2d.fit_transform(data)

    return proj_2d

#### clustering on original data

In [ ]:
centroids, kmeans_labels = sklearnkmeans(test)

for i in centroids:
    plt.plot(i)
plt.title("kmeans cluster centers original data")
plt.show()

In [ ]:
centroids, dtwkmeans_labels = k_means_dtw(X_test,num_clust=5,num_iter=10,w=5)

for i in centroids:
    plt.plot(i)
plt.title("dtw kmeans cluster centers original data")
plt.show()

In [ ]:
umap_2d = umapt(X_test)
plt.title("umap on original data with original labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=y_test, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on original data with kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on original data with dtw kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=dtwkmeans_labels, s=0.1, cmap='Spectral')

### LSTM


#### LSTM Model & Training

In [ ]:
#reshape to fit in lstm
train_lstm = np.reshape(train, train.shape + (1,))
test_lstm = np.reshape(test, test.shape + (1,))

train_lstm.shape

In [ ]:
# define model
model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(187,1)))
model.add(RepeatVector(187))
model.add(LSTM(30, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))

In [ ]:
model.compile(optimizer='adadelta', loss='binary_crossentropy')

history_lstm_seq = model.fit(train_lstm, train_lstm, epochs=200,
          batch_size=16,
          shuffle=True,
          steps_per_epoch=50,
          validation_steps=20,
          validation_data=(test_lstm, test_lstm))


In [ ]:
plt.plot(history_lstm_seq.history['loss'], label='train')
plt.plot(history_lstm_seq.history['val_loss'], label='test')
plt.title("loss LSTM seq")
plt.legend()
plt.show()

In [ ]:
predicted_lstm_seq = model.predict(test_lstm)
predicted_lstm_seq = np.reshape(predicted_lstm_seq, (2048, 187))

In [ ]:
plt.plot(predicted_lstm_seq[5])
plt.show()

#### Clustering

In [ ]:
centroids, kmeans_labels = sklearnkmeans(predicted_lstm_seq)

#plot centroids
for i in centroids:
    plt.plot(i)
plt.title("kmean cluster centers lstm")
plt.show()

In [ ]:
centroids, labels_dtwkmeans=k_means_dtw(predicted_lstm_seq,num_clust=5,num_iter=10,w=5)

for i in centroids:
    plt.plot(i)
plt.title("dtw kmeans cluster centers lstm embeddings")
plt.show()

In [ ]:
umap_2d = umapt(predicted_lstm_seq)
plt.title("umap on lstm embeddings with original labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=y_test, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on lstm embeddings with kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on lstm embeddings with dtw kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=dtwkmeans_labels, s=0.1, cmap='Spectral')

## Autoencoder

### Simple Autoencoder

#### Model & Training of Embeddings

In [ ]:
# size of encoded representation
encoding_dim = 15
#input sequence
input = Input(shape=(187,))
# encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input)
# lossy reconstruction of the input
decoded = Dense(187, activation='sigmoid')(encoded)

In [ ]:
autoencoder = Model(input, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder_ac = Model(input, encoded)
# This is our encoded input
encoded_input_ac = Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer_ac = autoencoder.layers[-1]
# Create the decoder model
decoder_ac = Model(encoded_input_ac, decoder_layer_ac(encoded_input_ac))

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

history_ac  = autoencoder.fit(train, train,
                epochs=1000,
                batch_size=16,
                shuffle=True,
                validation_data=(test, test))

In [ ]:
plt.plot(history_ac.history['loss'], label='train')
plt.plot(history_ac.history['val_loss'], label='test')
plt.title("loss simple autoencoder")
plt.legend()
plt.show()

In [ ]:
encoded_data_ac = encoder_ac.predict(test)
decoded_data_ac = decoder_ac.predict(encoded_data_ac)

#### Clustering

In [ ]:
centroids, kmeans_labels = sklearnkmeans(encoded_data_ac)

#plot centroids
for i in centroids:
    plt.plot(i)
plt.title("kmeans cluster centers simple autoencoder embeddings")
plt.show()

In [ ]:
centroids, dtwkmeans_labels =k_means_dtw(encoded_data_ac,num_clust=5,num_iter=10,w=5)

for i in centroids:
    plt.plot(i)
plt.title("dtw kmeans cluster centers simple autoencoder embeddings")
plt.show()

In [ ]:
umap_2d = umapt(encoded_data_ac)
print(umap_2d)
plt.title("umap on simple ac embeddings with original labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=y_test, s=0.1, cmap='Spectral')


In [ ]:
plt.title("umap on simple ac embeddings with kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on simple ac embeddings with dtw kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=dtwkmeans_labels, s=0.1, cmap='Spectral')

### Deep Autoencoder

#### Model & Training of Embeddings

In [ ]:
input = Input(shape=(187,))

encoded = Dense(128, activation='relu')(input)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(187, activation='sigmoid')(decoded)

In [ ]:
autoencoder_deep = Model(input, decoded)

# This model maps an input to its encoded representation
encoder_ac_deep = Model(input, encoded)
encoded_input_ac_deep = Input(shape=(187,))
#decoder_layer_ac_deep = autoencoder_deep.layers[-1]
#decoder_ac_deep = Model(encoded_input_ac_deep, decoder_layer_ac_deep(encoded_input_ac_deep))

In [ ]:
autoencoder_deep.compile(optimizer='adam', loss='binary_crossentropy')

history_ac_deep = autoencoder_deep.fit(train, train,
                epochs=200,
                batch_size=16,
                shuffle=True,
                validation_data=(test, test))

In [ ]:
plt.plot(history_ac_deep.history['loss'], label='train')
plt.plot(history_ac_deep.history['val_loss'], label='test')
plt.title("loss deep autoencoder")
plt.legend()
plt.show()

In [ ]:
encoded_data_ac_deep = encoder_ac_deep.predict(test)
#decoded_data_ac_deep = decoder_ac.predict(encoded_data_ac_deep)

#### Clustering

In [ ]:
centroids, kmeans_labels = sklearnkmeans(encoded_data_ac_deep)

#plot centroids
for i in centroids:
    plt.plot(i)
plt.title("kmean cluster centers deep autoencoder embeddings")
plt.show()

In [ ]:
centroids, dtwkmeans_labels = k_means_dtw(encoded_data_ac_deep,num_clust=5,num_iter=10,w=5)

for i in centroids:
    plt.plot(i)
plt.title("dtw kmeans cluster centers deep autoencoder embeddings")
plt.show()

In [ ]:
umap_2d = umapt(encoded_data_ac_deep)
plt.title("umap on deep ac embeddings with original labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=y_test, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on deep ac embeddings with kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on deep ac embeddings with dtw kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=dtwkmeans_labels, s=0.1, cmap='Spectral')

### LSTM autoencoder

#### Model & Training of Embeddings

In [ ]:
timesteps = 187
input_dim = 1
latent_dim = 10

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

In [ ]:
sequence_autoencoder = Model(inputs, decoded)

encoder_ac_lstm = Model(inputs, encoded)
encoded_ac_input = Input(shape=(encoding_dim,1))
#decoder_layer = sequence_autoencoder.layers[-1]
#decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
sequence_autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

history_ac_lstm = sequence_autoencoder.fit(train_lstm, train_lstm,
                epochs=50,
                batch_size=16,
                shuffle=True,
                validation_data=(test_lstm, test_lstm))

In [ ]:
plt.plot(history_ac_lstm.history['loss'], label='train')
plt.plot(history_ac_lstm.history['val_loss'], label='test')
plt.title("loss lstm autoencoder")
plt.legend()
plt.show()

In [ ]:
encoded_data_ac_lstm = encoder_ac_lstm.predict(test_lstm)

#### Clustering

In [ ]:
centroids, kmeans_labels = sklearnkmeans(encoded_data_ac_lstm)

#plot centroids
for i in centroids:
    plt.plot(i)
plt.title("kmean cluster centers lstm autoencoder embeddings")
plt.show()

In [ ]:
centroids, dtwkmeans_labels = k_means_dtw(encoded_data_ac_lstm,num_clust=5,num_iter=10,w=5)

for i in centroids:
    plt.plot(i)
plt.title("dtw kmeans cluster centers lstm autoencoder embeddings")
plt.show()

In [ ]:
umap_2d = umapt(encoded_data_ac_lstm)
plt.title("umap on lstm ac embeddings with original labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=y_test, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on lstm ac embeddings with kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral')

In [ ]:
plt.title("umap on simple ac embeddings with dtw kmeans labels")
plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=dtwkmeans_labels, s=0.1, cmap='Spectral')